In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader

from torchsummary import summary

from torchvision import datasets, transforms

In [2]:
from copy import deepcopy

import numpy as np
import pandas as pd

from termcolor import cprint

In [3]:
data_path = "/tmp/files/"

tensor_mnist = datasets.MNIST(
    data_path, train=True, download=True, transform=transforms.ToTensor()
)

tensor_images = torch.stack([tensor_image for tensor_image, _ in tensor_mnist], dim=3)

tensor_images.shape

torch.Size([1, 28, 28, 60000])

In [4]:
tensor_mean = tensor_images.view(1, -1).mean(dim=1)
tensor_mean

tensor([0.1307])

In [5]:
tensor_std = tensor_images.view(1, -1).std(dim=1)
tensor_std

tensor([0.3081])

In [6]:
training_dataset = datasets.MNIST(
    "/tmp/files/",
    train=True,
    download=True,
    transform=transforms.Compose(
        [
            transforms.ToTensor(),
            transforms.Normalize(tensor_mean, tensor_std),
        ]
    ),
)

validation_dataset = datasets.MNIST(
    "/tmp/files/",
    train=False,
    download=True,
    transform=transforms.Compose(
        [
            transforms.ToTensor(),
            transforms.Normalize(tensor_mean, tensor_std),
        ]
    ),
)

In [7]:
training_dataset.data.shape, training_dataset.targets.shape

(torch.Size([60000, 28, 28]), torch.Size([60000]))

In [8]:
training_dataset.data[0].shape, training_dataset.targets[0].item()

(torch.Size([28, 28]), 5)

In [9]:
training_df = pd.DataFrame(
    {
        "image": [training_dataset.data[i].numpy() for i in range(len(training_dataset))],
        "label": training_dataset.targets.numpy(),
    }
)

training_df.head()

,image,label
0,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",5
1,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",0
2,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",4
3,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",1
4,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",9


In [10]:
training_df['label'].sort_values().unique()

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

In [11]:
for i in training_df["label"].sort_values().unique():
    cprint(
        f"\nLabel {i} count: {len(training_df[training_df['label'] == i])}",
        "green",
        attrs=["bold"],
        end="\n" + "-" * 60 + "\n",
    )
    exec(f"training_df_{i} = deepcopy(training_df)")
    exec(f"training_df_{i}.loc[training_df_{i}['label'] != i, 'label'] = 0")
    exec(f"cprint(training_df_{i}.head(), 'magenta')")
    exec(f"cprint(training_df_{i}['label'].value_counts()\n, 'red')")


Label 0 count: 5923
------------------------------------------------------------
                                               image  label
0  [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...      0
1  [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...      0
2  [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...      0
3  [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...      0
4  [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...      0
label
0    60000
Name: count, dtype: int64

Label 1 count: 6742
------------------------------------------------------------
                                               image  label
0  [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...      0
1  [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...      0
2  [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...      0
3  [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...      1
4  [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...      0
label
0    53258
1     6742
Name: count, dtype: int64

Label 2 count: 595

In [12]:
if torch.backends.mps.is_available():
    cprint("MPS is available", "green")
    device = torch.device("mps:0")
elif torch.backends.cuda.is_available():
    cprint("CUDA is available", "green")
    device = torch.device("cuda:0")
elif torch.backends.cudnn.is_built():
    cprint("CUDNN is available", "green")
    device = torch.device("cuda:0")
else:
    cprint("CUDA and MPS are not available", "red")
    cprint("Using CPU", "red")
    device = torch.device("cpu")

MPS is available


In [13]:
training_loader = torch.utils.data.DataLoader(
    training_dataset, batch_size=100, shuffle=True
)

validation_loader = torch.utils.data.DataLoader(
    validation_dataset, batch_size=100, shuffle=False
)

In [14]:
for inputs, labels in training_loader:
    inputs = inputs.to(device)
    print(inputs.shape)
    labels = labels.to(device)
    print(labels.shape)
    break

torch.Size([100, 1, 28, 28])
torch.Size([100])
